# Гайд по CberEmissionTrack
Для начала, надо установить трекер(установка с помощью conda появится позже):

In [1]:
# во избежании путанницы на случай, если пакет устанавливался раньше, лучше удалить его и установить заново
!pip uninstall SberEmissionTrack -y
!pip install git+git://github.com/vladimir-laz/AIRIEmisisonTracker.git

Found existing installation: SberEmissionTrack 0.2.0
Uninstalling SberEmissionTrack-0.2.0:
  Successfully uninstalled SberEmissionTrack-0.2.0
  Cloning git://github.com/vladimir-laz/AIRIEmisisonTracker.git to /tmp/pip-req-build-m1q71neq
  Running command git clone -q git://github.com/vladimir-laz/AIRIEmisisonTracker.git /tmp/pip-req-build-m1q71neq
  Resolved git://github.com/vladimir-laz/AIRIEmisisonTracker.git to commit 028e9eb83f5cf5fdd73813d73045ba6254f637c4
  Created wheel for SberEmissionTrack: filename=SberEmissionTrack-0.2.0-py3-none-any.whl size=5864 sha256=525bcafa72a7a7914773bfbe8b169c87828820a6ee34d2b9183601678fc61e38
  Stored in directory: /tmp/pip-ephem-wheel-cache-ro86r3c8/wheels/67/e7/41/7ae3b1d2339881a7b0688acdc1bbcca48d205b78766f2ad749
Successfully built SberEmissionTrack


Делаем необходимые импорты

In [2]:
import SberEmissionTrack

Можно посмотреть, какие устройства доступны:

In [3]:
SberEmissionTrack.available_devices()

Seeable cpu devices:
        Intel(R) Xeon(R) Platinum 8168 CPU @ 2.70GHz: 96 devices
Seeable gpu devices:
        Tesla V100-SXM3-32GB: 2 devices


Соответственно, если нет доступных gpu, это будет обозначенно. Однако если не будет свободных cpu, то возникнет ошибка, ибо cpu всегда должен быть виден.

С автоматическим определением gpu проблем нет, но вот название cpu выводится не такое, как обычно указывается в официальных базах. В связи с этим полностью автоматические определение модели cpu требует либо составления собственной базы данных, либо тяжелой работы с регулярными выражениями, которая не факт что закончится успехом.
В связи с этим, при создании трекера и вызове метода .start() пользователю будет выведена модель cpu и будет предложено ввести его TDP самостоятельно. Попробуйте запустить и проверить(для "Intel(R) Xeon(R) Platinum 8168 CPU @ 2.70GHz." TDP = 205Ватт):

In [4]:
tracker = SberEmissionTrack.Tracker(project_name="TEST",
                                    experiment_description="testing AIRI tracker",
                                    save_file_name="emission.csv",
                                    measure_period=1)
tracker.start()
# здесь мог быть ваш код
tracker.stop()

Name of your cpu: Intel(R) Xeon(R) Platinum 8168 CPU @ 2.70GHz.
Please, enter it's TDP(watt): 205


Конечно, постоянно вводить значение TDP - неудобно, поэтому если оно известно, то можно ввести его заранее:

In [6]:
tracker = SberEmissionTrack.Tracker(project_name="TEST",
                                    experiment_description="testing AIRI tracker",
                                    save_file_name="emission.csv",
                                    measure_period=1, 
                                    cpu_tdp=205)
tracker.start()
# здесь мог быть ваш код
tracker.stop()

При запуске метода .start() идет подсчет базового(фонового) энергопотребления. Для этого желательно брать не текущие значения, а усредненые, это занимает около 10 секунд. 

Также если расчетов несколько, то рекомендуется подождать между ними около 20 секунд. За это время энергопотребление успевает упасть до фонового

Также лучше использовать не очень большой measure_period

Если вдруг что-то непонятно с работой трекера, то всегда можно обратиться к функции help(). Там есть описание класса, описание функций и пример использования

In [7]:
help(SberEmissionTrack.Tracker)

Help on class Tracker in module SberEmissionTrack.emission_track:

class Tracker(builtins.object)
 |  Tracker(project_name, experiment_description, save_file_name='emission.csv', measure_period=2, emission_level=511.7942, cpu_tdp=None)
 |  
 |  In order to correctly calculate gpu or cpu ppower consumption you should create
 |  Tracker before any gpu and cpu uses as tracker considers background gpu and cpu power
 |  
 |  For every new gpu calculation it should be created new tracker
 |  
 |  ----------------------------------------------------------------------
 |  Use example:
 |  
 |  import SberEmissionTrack.Tracker
 |  tracker = Tracker(project_name=your_project_name,
 |                    experiment_description=your_experiment_description,
 |                    save_file_name="you_file_name",
 |                    measure_period=2,   #measurement will be done every 2 seconds
 |                    )
 |  tracker.start()
 |  *your gpu calculations*
 |  tracker.stop()
 |  
 |  --------